# Forest Fire Spreading 


## Background

Wildfires are one of the most harmful phenomena in Chile burning thousands of forest hectares every year destroying significant flora and fauna, and affecting the air, water cycles, and the ecosystem. Wildfires start by varied reasons, ranging from reckless human behavior to extreme weather and environmental conditions

## Main Goal

Build a mathematical model based on SPDE qualitatively realistic with the phenomenon behaviour. 

## Objectives
1. Start with an agent based model, working with the weather conditions and initial fires focus. 
2. Get a continuous model based on SPDE from the ABM.
3. Assess wind relevance to the model.

In [1]:
import firemodels.cellularautomata as ca
import firemodels.cellularautomatalayers as cal
import firemodels.temperature as temp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

## Cellular Automata

In [ ]:
def fireFocus(M, N, i, j, size):
    focus = np.zeros((M, N))
    focus[i-size:i+size, j-size:j+size] = np.ones((2*size, 2*size)) 
    return focus

In [ ]:
def plotStates(t=0):
    play = widgets.Play(
        value=0,
        min=0,
        max=len(states)-1,
        step=1,
        description="Press play",
        disabled=False
    )
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    widgets.HBox([play, slider])
    widgets.interact(automata.plotStates, t=play)

In [ ]:
# Testing
M = 101
N = 101
initial = fireFocus(M, N, 50, 50, 2)
rule = .35
#neighborhood = 'vonneumann'
neighborhood = 'moore'
times = 100

In [ ]:
automata = ca.new(M, N, initial, rule, neighborhood)
states = automata.propagate(times)

In [ ]:
plotStates()

# Cellular Automata with layers

In [ ]:
temperature = np.load('data/temperature100x100.npy')
wind_speed = np.load('data/wind_speed100x100.npy')
humidity = np.load('data/humidity100x100.npy')
pressure = np.load('data/pressure100x100.npy')

In [ ]:
temperature = temperature / np.max(temperature)
wind_speed = wind_speed / np.max(wind_speed)
humidity = humidity / np.max(humidity)
pressure = pressure / np.max(pressure)

In [ ]:
# Testing
(M, N) = temperature.shape
wd = np.ones_like(temperature)*0
world = [temperature, wind_speed, wd, humidity, pressure]
initial = fireFocus(M, N, 50, 50, 2)
#neighborhood = 'vonneumann'
neighborhood = 'moore'
alpha = .5
beta = 1-alpha
times = 200
rule = .3

In [ ]:
automata = cal.new(initial, world, neighborhood, rule, alpha, beta)
states = automata.propagate(times)

In [ ]:
plotStates()

* $BT$: Burning trees
* $T$: Temperature

<!--
\begin{equation}
    BT_{t+1} = BT_{t} + F(BT_t, T_t, W_t, T_t, P_T)
\end{equation}
\begin{equation}
    T_{t+1} = F_t +  G(BT_t, T_t, W_t, T_t, P_T)
\end{equation}
\begin{equation}
    W_{t+1} = W_t +  I( W_t)
\end{equation}
-->

## Temperature

### Discrete

\begin{equation}
    \begin{split}
        T_{i,j}^{k+1} & = T_{i,j}^k - \frac45 T_{i,j}^k 
            + \frac15\left(T_{i-1,j}^k+T_{i+1,j}^k+T_{i,j-1}^k+T_{i,j+1}^k\right) \\
            & = \frac15\left(T_{i-1,j}^k+T_{i+1,j}^k+T_{i,j}^k+T_{i,j-1}^k+T_{i,j+1}^k\right)
    \end{split}
\end{equation}

In [2]:
def temperatureFocus(M, N):
    x = np.linspace(0, 1, N)
    y = np.linspace(0, 1, M)
    X, Y = np.meshgrid(x, y)
    return 1e3*np.exp(-1000*((X-.5)**2 + (Y-.5)**2))

In [3]:
M, N = 100, 100
initial = temperatureFocus(M, N)
times = 100
c = 1/5

In [4]:
dte = temp.discrete(c, 0, times, initial)
dtemps = dte.propagate()

In [5]:
widgets.interact(dte.plotTemperatures, t=widgets.IntSlider(min=0, max=len(dtemps)-1, continuous_update=False),
                 temperatures=widgets.fixed(dtemps))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=99), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### With Forest

In [6]:
def temperatureFocus(M, N):
    temperature = np.zeros((M,N))
    A = np.zeros((M,N))
    A[M//2, N//2] = 1
    A[M//2+1, N//2] = 1
    temperature = temperature + A * 600
    return temperature, A

In [7]:
def temperatureFocus2(M, N):
    # Initial trees burning
    tree = np.zeros((M,N))
    tree[M//2, N//2] = 1
    tree[M//2+1, N//2] = 1
    # Initial temperature
    #x = np.linspace(0, 1, N)
    #y = np.linspace(0, 1, M)
    #X, Y = np.meshgrid(x, y)
    #temperatures = 1e3*np.exp(-1000*((X-.5)**2 + (Y-.5)**2))
    temperature = np.zeros((M,N))
    temperature = temperature + tree * 600
    fuel = np.zeros((M,N))
    radius = 10
    cx, cy = M//2, N//2 # The center of circle
    y, x = np.ogrid[-radius: radius, -radius: radius]
    index = x**2 + y**2 <= radius**2
    fuel[cy-radius:cy+radius, cx-radius:cx+radius][index] = 1
    return temperature, tree, fuel

In [8]:
M, N = 100, 100
#initial, A = temperatureFocus(M, N)
T0, A0, Y0 = temperatureFocus2(M, N)
times = 300
c = 1/5
b = 8
maxTemp = 1000

In [9]:
dt2 = temp.discrete(c, 0, times, T0, A0, np.zeros_like(T0), None, b, maxTemp)

In [10]:
dt2a_temps, dt2a_As, dt2a_fuels = dt2.propagate()

In [11]:
widgets.interact(dt2.plotTemperatures, t=widgets.IntSlider(min=0, max=len(dt2a_temps)-1, 
    continuous_update=False), temperatures=widgets.fixed(dt2a_temps))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### Stochastic

\begin{equation}
    T_{i,j}^{k+1} = T_{i,j}^{k} - 4 \, c \, T_{i,j}^{k} + (c + \sigma_1 \Delta  \, Z^k_{i,j}) \left(T_{i+1,j}^{k} + T_{i-1,j}^{k} + T_{i,j+1}^{k} + T_{i,j-1}^{k}\right) + \sigma_2 \Delta W^k_{i,j}
\end{equation}

#### Approach 1 $\sigma_1=0$

In [12]:
dt2b_temps, dt2b_A, _ = dt2.propagate(None, 5)

In [13]:
widgets.interact(dt2.plotTemperatures, t=widgets.IntSlider(min=0, max=len(dt2b_temps)-1, 
    continuous_update=False), temperatures=widgets.fixed(dt2b_temps))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### Approach 2 $\sigma_1, \sigma_2 \neq 0$

In [14]:
dt2c_temps, dt2c_A, _ = dt2.propagate(1/6, 1)

In [15]:
widgets.interact(dt2.plotTemperatures, t=widgets.IntSlider(min=0, max=len(dt2c_temps)-1, 
    continuous_update=False), temperatures=widgets.fixed(dt2c_temps))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### Fuel added

\begin{equation}
    \begin{split}
        T_{i,j}^{k+1} & = T_{i,j}^{k} - 4 \, c \, T_{i,j}^{k} + (c + \sigma_1 \Delta  \, Z^k_{i,j}) \left(T_{i+1,j}^{k} + T_{i-1,j}^{k} + T_{i,j+1}^{k} + T_{i,j-1}^{k}\right) + \sigma_2 \Delta W^k_{i,j} + H\, A^k_{i,j}\,Y_{i,j}^k \, Z  \exp(E/T^k_{i,j}) \\
        Y^{k+1}_{i,j} & = Y^k_{i,j} - \,Y_{i,j}^k \, Z  \exp(E/T^k_{i,j})
    \end{split}
\end{equation}

#### With Forest and Fuel $\sigma_1=\sigma_2=0$

In [16]:
Ea = 1
Z = .1
H = 5500
b = 8
maxTemp = 1000
dt3 = temp.discrete(c, 0, times, T0, A0, Y0, None, b, maxTemp, Ea, Z, H)
dt3a_temps, dt3a_As, dt3a_fuel = dt3.propagate()#4/30, 20)

In [17]:
widgets.interact(dt3.plotSimulation, t=widgets.IntSlider(min=0, max=len(dt3a_temps)-1, continuous_update=False), 
                 temperatures=widgets.fixed(dt3a_temps), fuels=widgets.fixed(dt3a_fuel), trees=widgets.fixed(dt3a_As))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### With Forest and Fuel Stochastic  $\sigma_1, \sigma_2 \neq 0$

In [18]:
dt3_2 = temp.discrete(c, 0, times, T0, A0, Y0, None, b, maxTemp, Ea*1e-5, Z, H)#temp.discrete(c, initial, times, A, b, maxTemp, Ea*1e-5, Z, H)
dt3b_temps, dt3b_As, dt3b_fuel = dt3_2.propagate(1/10, 20)

In [19]:
widgets.interact(dt3.plotSimulation, t=widgets.IntSlider(min=0, max=len(dt3b_temps)-1, 
    continuous_update=False), temperatures=widgets.fixed(dt3b_temps), fuels=widgets.fixed(dt3b_fuel), trees=widgets.fixed(dt3b_As))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### With wind

\begin{equation}
    \begin{split}
        T_{i,j}^{k+1} & = T_{i,j}^{k} - 4 \, c \, T_{i,j}^{k} + (c + \sigma_1 \Delta  \, Z^k_{i,j}) 
            \left(T_{i+1,j}^{k} + T_{i-1,j}^{k} + T_{i,j+1}^{k} + T_{i,j-1}^{k}\right) 
            + \gamma(V_x+V_y)+ \sigma_2 \Delta W^k_{i,j} + H\, A^k_{i,j} \,Y_{i,j}^k \, Z  \exp(-E/T^k_{i,j}) \\
        Y^{k+1}_{i,j} & = Y^k_{i,j} - \,Y_{i,j}^k \, Z  \exp(-E/T^k_{i,j})
    \end{split}
\end{equation}

Where:

$$V_x=\begin{cases}-v_x(T_{i,j}^k - T_{i-1,j}^k) & \textrm{if } v_x \geq 0 \\-v_x(T_{i+1,j}^k - T_{i,j}^k) & \textrm{otherwise}\end{cases}$$

$$V_y=\begin{cases}-v_y(T_{i,j}^k - T_{i,j-1}^k) & \textrm{if } v_y \geq 0 \\-v_y(T_{i,j+1}^k - T_{i,j}^k) & \textrm{otherwise}\end{cases}$$

In [20]:
gamma = 1
V = [(-.1, -.1)]
dtempv = temp.discrete(c, gamma, times, T0, A0, Y0, V, b, maxTemp, Ea*1e-3, Z, H) 
dtempvs, Asv, fuelsv = dtempv.propagate()#1/13, 15)

In [21]:
widgets.interact(dtempv.plotSimulation, t=widgets.IntSlider(min=0, max=len(dtempvs)-1, 
    continuous_update=False), temperatures=widgets.fixed(dtempvs), fuels=widgets.fixed(fuelsv), trees=widgets.fixed(Asv))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

#### Newton's law of cooling

\begin{equation}
    \begin{split}
        T_{i,j}^{k+1} & = T_{i,j}^{k} - 4 \, c \, T_{i,j}^{k} + (c + \sigma_1 \Delta  \, Z^k_{i,j}) 
            \left(T_{i+1,j}^{k} + T_{i-1,j}^{k} + T_{i,j+1}^{k} + T_{i,j-1}^{k}\right)
            + \gamma(V_x+V_y) + H\, A^k_{i,j} \,Y_{i,j}^k \, Z  \exp(-E/T^k_{i,j}) 
            \, - h(T_{i,j}^k - T_{env}) + \sigma_2 \Delta W^k_{i,j} \\
        Y^{k+1}_{i,j} & = Y^k_{i,j} - \,Y_{i,j}^k \, Z  \exp(-E/T^k_{i,j})
    \end{split}
\end{equation}

Where:

$$V_x=\begin{cases}-v_x(T_{i,j}^k - T_{i-1,j}^k) & \textrm{if } v_x \geq 0 \\-v_x(T_{i+1,j}^k - T_{i,j}^k) & \textrm{otherwise}\end{cases}$$

$$V_y=\begin{cases}-v_y(T_{i,j}^k - T_{i,j-1}^k) & \textrm{if } v_y \geq 0 \\-v_y(T_{i,j+1}^k - T_{i,j}^k) & \textrm{otherwise}\end{cases}$$

In [22]:
Ea = 1
h = 1e-2
T_ref = 30
dtenv = temp.discrete(c, gamma, times, T0, A0, Y0, V, b, maxTemp, Ea, Z, H, h, T_ref) 
dtsenv, Asenv, fsenv = dtenv.propagate()

In [23]:
widgets.interact(dtempv.plotSimulation, t=widgets.IntSlider(min=0, max=len(dtsenv)-1, 
    continuous_update=False), temperatures=widgets.fixed(dtsenv), fuels=widgets.fixed(fsenv), trees=widgets.fixed(Asenv))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=299), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

### Continuous

\begin{equation}
    \partial T(i,j,k)= (c' \, \partial t + \sigma_1' \, \partial Z_{i,j}(t))\, \Delta T(i,j,k) + \sigma_2' \,\partial W_{i,j}(t)
\end{equation}

#### With $\sigma_1=\sigma_2=0$

In [ ]:
def temperatureCFocus(M, N):
    x = np.linspace(0, 1, N)
    y = np.linspace(0, 1, M)
    X, Y = np.meshgrid(x, y)
    return 1e3*np.exp(-1000*((X-.5)**2 + (Y-.5)**2))

In [ ]:
M, N = 50, 50  

initial, A = temperatureFocus(M, N)

# Parameters
mu = 1/5 
T = 1000
dt = 1e-4
b = 8
maxTemp = 1000

In [ ]:
ct = temp.continuous(initial, mu, dt, T, b, maxTemp, A=A)
cta, Aa, Wa = ct.solvePDE()

In [ ]:
widgets.interact(ct.plotTemperatures, t=widgets.IntSlider(min=0, max=len(cta)-1), 
                 temperatures=widgets.fixed(cta))

#### Approach 1 $\sigma_1\neq 0$

In [ ]:
ctb, Ab, Wb = ct.solvePDE(1/3)

In [ ]:
widgets.interact(ct.plotTemperatures, t=widgets.IntSlider(min=0, max=len(ctb)-1), 
                 temperatures=widgets.fixed(ctb))

#### Approach 1 $\sigma_1,\sigma_2\neq 0$

In [ ]:
ctc, Ac, Wc = ct.solvePDE(1/8, 2000)

In [ ]:
widgets.interact(ct.plotTemperatures, t=widgets.IntSlider(min=0, max=len(ctc)-1), 
                 temperatures=widgets.fixed(ctc))

## Topography

In [ ]:
import csv

In [ ]:
def CSVElevation(path):
    x, y, z = [], [], []
    with open(path, newline='') as csvfile:
        data = csv.reader(csvfile, delimiter='\t', quotechar='|')
        next(data, None)
        for row in data:
            y.append(float(row[1]))
            x.append(float(row[2]))
            z.append(float(row[3]))
        
    return np.array(x), np.array(y), np.array(z)

In [ ]:
M, N = 100, 100

start_lat, start_lng = -33.088309, -71.655849
end_lat, end_lng = -33.141629, -71.594160

In [ ]:
x, y, z = CSVElevation("data/valpo_elevation.csv")

In [ ]:
X = x.reshape(M, N)
Y = y.reshape(M, N)
Z = z.reshape(M, N)

In [ ]:
plt.contourf(X, Y, Z)
plt.colorbar()
plt.show()

In [ ]:
# Compute gradient
dx = 1.#X[0, 1] - X[0, 0]
dy = 1.#Y[0, 1] - Y[0, 0]
#Z_x = np.gradient(Z, dx, axis=1)
#Z_y = np.gradient(Z, dy, axis=0)
Z_x = np.gradient(Z, axis=1)
Z_y = np.gradient(Z, axis=0)
plt.contourf(X, Y, Z_x)
plt.colorbar()
plt.show()
plt.contourf(X, Y, Z_y)
plt.colorbar()
plt.show()

In [ ]:
p = 4

nZ_x = np.zeros((M//p, N//p))
nZ_y = np.zeros((M//p, N//p))

for i in range(M//p):
    for j in range(N//p):
        nZ_x[i, j] = np.sum(Z_x[p*i:p*(i+1), p*j:p*(j+1)]) / (p**2)
        nZ_y[i, j] = np.sum(Z_y[p*i:p*(i+1), p*j:p*(j+1)]) / (p**2)
    
#elev = plt.contourf(X[::s,::s], Y[::s,::s], Z[::s,::s], alpha=.5)
plt.figure(figsize=(12, 8))
elv = plt.contourf(X[::p,::p], Y[::p,::p], Z[::p,::p], alpha=.5)
plt.quiver(X[::p,::p], Y[::p,::p], nZ_x, nZ_y)
plt.colorbar(elv)
plt.show()

In [ ]:
s = 2
plt.figure(figsize=(12, 8))
elev = plt.contourf(X[::s,::s], Y[::s,::s], Z[::s,::s], alpha=.5)
plt.quiver(X[::s,::s], Y[::s,::s], Z_x[::s,::s], Z_y[::s,::s])
plt.colorbar(elev)
plt.show()

## Wind

In [24]:
from scipy import interpolate

In [25]:
v_spe = np.load('data/speed.npy')
v_dir = np.load('data/direction.npy')

In [194]:
T = 50
speed = v_spe[-T:]#v_spe[:T]
direction = np.radians(v_dir[-T:])
t = np.linspace(0, 10*T, T)
#plt.plot(t, speed)
#plt.plot(t, direction)
#plt.show()

fs = interpolate.interp1d(t, speed, kind='cubic')
fd = interpolate.interp1d(t, direction, kind='cubic')
fine_t = np.linspace(0, 10*T, 10*T)
int_speed = fs(fine_t)
int_direction = fd(fine_t)
#plt.plot(fine_t, int_speed)
#plt.plot(fine_t, int_direction)
#plt.show()

xv = int_speed * np.cos(int_direction)
yv = int_speed * np.sin(int_direction)
#plt.plot(xv)
#plt.plot(yv)
#plt.show()

In [195]:
ONES = np.ones((10, 10))

In [196]:
def plotWind(i, u, v):
    plt.quiver(u[i]*ONES, v[i]*ONES)
    plt.show()

In [197]:
widgets.interact(plotWind, i=widgets.IntSlider(min=0, max=len(xv)-1), u=widgets.fixed(xv), v=widgets.fixed(yv))

interactive(children=(IntSlider(value=0, description='i', max=499), Output()), _dom_classes=('widget-interact',))

<function __main__.plotWind>

In [198]:
winds = list()
for i in range(len(xv)):
    winds.append((1e-2*xv[i], 1e-2*yv[i]))

In [211]:
Ea = 1
h = 1e-2
T_ref = 30
Z = .05
H = 8000
c = 1/10
#wind2 = [(-.05,-.05)]
Y1 = np.ones_like(Y0)
dtenv = temp.discrete(c, gamma, times+200, T0, A0, Y1, winds[0:], b, maxTemp, Ea, Z, H, h, T_ref) 
dtsenv, Asenv, fsenv = dtenv.propagate()

In [212]:
widgets.interact(dtempv.plotSimulation, t=widgets.IntSlider(min=0, max=len(dtsenv)-1, 
    continuous_update=False), temperatures=widgets.fixed(dtsenv), fuels=widgets.fixed(fsenv), trees=widgets.fixed(Asenv))

interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=499), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>